# Use Close-Form Policy to Play MoutainCar-v0

In [1]:
import sys
import logging
import imp
import itertools

import numpy as np
np.random.seed(0)
import gym

imp.reload(logging)
logging.basicConfig(level=logging.DEBUG,
        format='%(asctime)s [%(levelname)s] %(message)s',
        stream=sys.stdout, datefmt='%H:%M:%S')

In [2]:
env = gym.make('MountainCar-v0')
env.seed(0)
for key in vars(env):
    logging.info('%s: %s', key, vars(env)[key])

11:20:53 [INFO] env: <MountainCarEnv<MountainCar-v0>>
11:20:53 [INFO] action_space: Discrete(3)
11:20:53 [INFO] observation_space: Box(2,)
11:20:53 [INFO] reward_range: (-inf, inf)
11:20:53 [INFO] metadata: {'render.modes': ['human', 'rgb_array'], 'video.frames_per_second': 30}
11:20:53 [INFO] _max_episode_steps: 200
11:20:53 [INFO] _elapsed_steps: None


In [3]:
class CloseFormAgent:
    def __init__(self, _):
        pass

    def reset(self, mode=None):
        pass

    def step(self, observation, _reward, _done):
        position, velocity = observation
        lb = min(-0.09 * (position + 0.25) ** 2 + 0.03,
                0.3 * (position + 0.9) ** 4 - 0.008)
        ub = -0.07 * (position + 0.38) ** 2 + 0.07
        if lb < velocity < ub:
            action = 2 # push right
        else:
            action = 0 # push left
        return action

    def close(self):
        pass


agent = CloseFormAgent(env)

In [4]:
def play_episode(env, agent, max_episode_steps=None, mode=None, render=False):
    observation, reward, done = env.reset(), 0., False
    agent.reset(mode=mode)
    episode_reward, elapsed_steps = 0., 0
    while True:
        action = agent.step(observation, reward, done)
        if render:
            env.render()
        if done:
            break
        observation, reward, done, _ = env.step(action)
        episode_reward += reward
        elapsed_steps += 1
        if max_episode_steps and elapsed_steps >= max_episode_steps:
            break
    agent.close()
    return episode_reward, elapsed_steps


logging.info('==== test ====')
episode_rewards = []
for episode in range(100):
    episode_reward, elapsed_steps = play_episode(env, agent)
    episode_rewards.append(episode_reward)
    logging.debug('test episode %d: reward = %.2f, steps = %d',
            episode, episode_reward, elapsed_steps)
logging.info('average episode reward = %.2f ± %.2f',
        np.mean(episode_rewards), np.std(episode_rewards))

11:20:53 [INFO] ==== test ====
11:20:53 [DEBUG] test episode 0: reward = -105.00, steps = 105
11:20:53 [DEBUG] test episode 1: reward = -84.00, steps = 84
11:20:53 [DEBUG] test episode 2: reward = -133.00, steps = 133
11:20:53 [DEBUG] test episode 3: reward = -105.00, steps = 105
11:20:53 [DEBUG] test episode 4: reward = -105.00, steps = 105
11:20:53 [DEBUG] test episode 5: reward = -100.00, steps = 100
11:20:53 [DEBUG] test episode 6: reward = -104.00, steps = 104
11:20:53 [DEBUG] test episode 7: reward = -100.00, steps = 100
11:20:53 [DEBUG] test episode 8: reward = -105.00, steps = 105
11:20:54 [DEBUG] test episode 9: reward = -145.00, steps = 145
11:20:54 [DEBUG] test episode 10: reward = -105.00, steps = 105
11:20:54 [DEBUG] test episode 11: reward = -83.00, steps = 83
11:20:54 [DEBUG] test episode 12: reward = -103.00, steps = 103
11:20:54 [DEBUG] test episode 13: reward = -156.00, steps = 156
11:20:54 [DEBUG] test episode 14: reward = -84.00, steps = 84
11:20:54 [DEBUG] test epi

In [5]:
env.close()